https://adventofcode.com/2018/day/15

In [1]:
from dataclasses import dataclass

import networkx as nx
from toolz import concat

In [2]:
with open('data/15-1.txt') as fh:
    data = fh.read()

In [3]:
print(data)

################################
#####################...########
###################....G########
###################....#########
#######.##########......########
#######G#########........#######
#######G#######.G.........######
#######.######..G.........######
#######.......##.G...G.G..######
########..##..#....G......G#####
############...#.....G.....#####
#...#######..........G.#...#####
#...#######...#####G......######
##...######..#######G.....#.##.#
###.G.#####.#########G.........#
###G..#####.#########.......#.E#
###..######.#########..........#
###.......#.#########.....E..E.#
#####G...#..#########.......#..#
####.G.#.#...#######.....G.....#
########......#####...........##
###########..................###
##########.................#####
##########.................#####
############..E.........E.....##
############.........E........##
###############.#............E##
##################...E..E..##.##
####################.#E..####.##
################.....######...##
##########

In [4]:
@dataclass(repr=True, order=True, frozen=True)
class Pt:
    row: int
    col: int

    @property
    def neighbors(self):
        """Sorted neighbor coords"""
        return [
            Pt(self.row - 1, self.col),
            Pt(self.row, self.col - 1),
            Pt(self.row, self.col + 1),
            Pt(self.row + 1, self.col)
        ]


@dataclass(repr=True)
class Fighter:
    team: str
    hp: int = 200
    ap: int = 3
    alive: bool = True
    
    
class GameOver(Exception):
    pass

In [5]:
def load_data(data):
    cave = nx.Graph()
    fighters = {}
    for i, line in enumerate(data.split('\n')):
        for j, c in enumerate(line):
            if c in ('.', 'G', 'E'):
                pt = Pt(i, j)
                cave.add_node(pt)
                if c in ('G', 'E'):
                    fighters[pt] = Fighter(c)
    
    for pt in cave:
        for nabe in pt.neighbors:
            if nabe in cave:
                cave.add_edge(pt, nabe)
    
    return cave, fighters


In [6]:
cave, fighters = load_data(data)

In [12]:
def turn(mypt, cave, fighters):
    try:
        me = fighters[mypt]
    except KeyError:
#         print(mypt, "can't move, is already dead")
        return
    enemy_team = 'G' if me.team == 'E' else 'E'
    enemies = {k:v for k, v in fighters.items() if v.team == enemy_team}
    if not enemies:
        raise GameOver

    def attack(attackerpt, defenderpt):
        attacker, defender = fighters[attackerpt], fighters[defenderpt]
        defender.hp -= attacker.ap
        if defender.hp <= 0:
            defender.alive = False
            del fighters[defenderpt]
            del enemies[defenderpt]

    attackable = [(enemies[nabe].hp, nabe) for nabe in mypt.neighbors if nabe in enemies and nabe in cave]        
    if attackable:
        _, defenderpt = min(attackable)
        attack(mypt, defenderpt)
        return
        
    other_fighters = {k for k in fighters if k != mypt}
    attack_pts = set(ap for ap in set(concat(e.neighbors for e in enemies))
                    if ap in cave and ap not in other_fighters)
    if not attack_pts:
        return

    cavecopy = cave.copy()
    cavecopy.remove_nodes_from(other_fighters)
        
    attack_pt_distances = []
    for ap in attack_pts:
        try:
            d = nx.shortest_path_length(cavecopy, mypt, ap)
        except nx.NetworkXNoPath:
            continue
        attack_pt_distances.append((d, ap))
    if not attack_pt_distances:
        return
    attack_pt_distances.sort()
    _, attack_pt = min(attack_pt_distances)

    nabe_attack_pt_distances = []
    for nabe in mypt.neighbors:
        if nabe not in cavecopy:
            continue
        try:
            d = nx.shortest_path_length(cavecopy, nabe, attack_pt)
        except nx.NetworkXNoPath:
            continue
        nabe_attack_pt_distances.append((d, nabe))
    _, moveto = min(nabe_attack_pt_distances)
    
    fighters[moveto] = fighters[mypt]
    del fighters[mypt]
    mypt = moveto

    attackable = [(enemies[nabe].hp, nabe) for nabe in mypt.neighbors if nabe in enemies and nabe in cave]        
    if attackable:
        _, defenderpt = min(attackable)
        attack(mypt, defenderpt)


In [8]:
def round(cave, fighters):
    for pt in sorted(fighters.keys()):
        turn(pt, cave, fighters)

In [9]:
sorted(fighters.keys())

[Pt(row=2, col=23),
 Pt(row=5, col=7),
 Pt(row=6, col=7),
 Pt(row=6, col=16),
 Pt(row=7, col=16),
 Pt(row=8, col=17),
 Pt(row=8, col=21),
 Pt(row=8, col=23),
 Pt(row=9, col=19),
 Pt(row=9, col=26),
 Pt(row=10, col=21),
 Pt(row=11, col=21),
 Pt(row=12, col=19),
 Pt(row=13, col=20),
 Pt(row=14, col=4),
 Pt(row=14, col=21),
 Pt(row=15, col=3),
 Pt(row=15, col=30),
 Pt(row=17, col=26),
 Pt(row=17, col=29),
 Pt(row=18, col=5),
 Pt(row=19, col=5),
 Pt(row=19, col=25),
 Pt(row=24, col=14),
 Pt(row=24, col=24),
 Pt(row=25, col=21),
 Pt(row=26, col=29),
 Pt(row=27, col=21),
 Pt(row=27, col=24),
 Pt(row=28, col=22)]

In [10]:
testdata = """\
#######
#G......#
#.E.#...#
#..##..G#
#...##..#
#...#...#
#.G...G.#
#.....G.#
#######"""

In [13]:
(cave, fighters) = load_data(data)

for i in range(1_000_000):
    try:
        round(cave, fighters)
    except GameOver:
        hps = sum(v.hp for v in fighters.values() if v.alive)
        print(i, hps, i * hps)
        break
else:
    print(i)

82 2459 201638


195975 is too high